In [2]:
import pandas as pd
from dateutil.relativedelta import relativedelta
from datetime import datetime, timedelta, date

def load_data():
    neutor_xlsx = pd.read_excel('/Users/danielzellner/Documents/Studium/Bachelorthesis/src/Zaehlstelle_Neutor_2020_Stundenauswertung.xlsx', 
                                sheet_name=None, 
                                skiprows=2,
                                skipfooter=1,
                                engine='openpyxl')

    df_neutor = pd.concat(neutor_xlsx.values(), ignore_index=True)
    
    #df_neutor['Datum'] = datetime.strftime(df_neutor['Zeit'], '%Y-%m-%d')
    df_neutor['Datum'] = pd.to_datetime(df_neutor['Zeit'], format='%Y-%m-%d').dt.date.astype(str)
    df_neutor['Zeit'] = pd.to_datetime(df_neutor['Zeit']).dt.time
    
    df_neutor = df_neutor.drop(columns='Unnamed: 0')
    
    return df_neutor
    
def neutor_weekday():
    
    df_neutor = load_data()
    
    # Variable fuer Neutor und mean
    
    df_neutor_weekday = df_neutor.groupby('Datum', dropna=False).agg({'Neutor': 'sum', 
                                                                      'Neutor FR stadteinwärts': 'sum',
                                                                      'Neutor FR stadtauswärts': 'sum'})
    df_neutor_weekday.index = pd.to_datetime(df_neutor_weekday.index)
    df_neutor_weekday['Wochentag'] = df_neutor_weekday.index.day_name()
    
    df_neutor_weekday = df_neutor_weekday.reset_index()
    
    return df_neutor_weekday

def neutor_last_week():
    df_neutor_weekday = neutor_weekday()
    
    to_day = date.today()
    new_to_day = to_day + relativedelta(years=-3)
    last_week = date.today() - timedelta(days=7)
    new_date = last_week + relativedelta(years=-3)

    new_date = pd.to_datetime(new_date)
    last_week = pd.to_datetime(last_week)
    new_to_day = pd.to_datetime(new_to_day)


    df_neutor_last_week = df_neutor_weekday.loc[
        (df_neutor_weekday['Datum'] >= new_date) &
        (df_neutor_weekday['Datum'] < last_week) &
        (df_neutor_weekday['Datum'] < new_to_day)
    ]

    return df_neutor_last_week

if __name__ == "__main__":
    df_neutor = load_data()
    df_neutor_weekday = neutor_weekday()
    df_neutor_last_week = neutor_last_week()

In [ ]:
fig = go.Figure(
    data=[
        go.Bar(
            x=df_neutor_last_week['Datum'],
            y=df_neutor_last_week['Neutor FR stadteinwärts'],
            offsetgroup=0
        ),
        go.Bar(
            x=df_neutor_last_week['Datum'],
            y=df_neutor_last_week['Neutor FR stadtauswärts'],
            offsetgroup=1
        )
    ]
)
fig.show()

TypeError: 'NoneType' object is not subscriptable

In [ ]:
print(df_neutor_last_week)

None


In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.express as px
import dash
from dash import Dash, dcc, html, Input, Output

df_neutor_weekday = neutor_weekday()
df_neutor_last_week = neutor_last_week()

app = Dash(__name__)

app.layout = html.Div(children=[
    html.Div([ 
        dcc.Graph(id='tip-graph'),
        dcc.Graph(id='second-graph'),  # Neuer Graph
        ]),
    ])

@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('yaxis-column', 'value'))
def update_figure(selected_column):
    fig = px.box(df_neutor_weekday, x='Wochentag', y=selected_column, points='all')
    #fig.update_layout(transition_duration=500)
    return fig

@app.callback(
    Output('second-graph', 'figure'))  # Callback-Methode für den zweiten Plot
def update_second_figure():
    fig = go.Figure()

    fig = fig.add_trace(
                go.Bar(
                    name='SE',
                    x=df_neutor_last_week['Datum'],
                    y=df_neutor_last_week['Neutor FR stadteinwärts'],
                    offsetgroup=0
                )
        )
    fig.add_trace(
            go.Bar(
                    name='SA',
                    x=df_neutor_last_week['Datum'],
                    y=df_neutor_last_week['Neutor FR stadtauswärts'],
                    offsetgroup=0,
                    base=df_neutor_last_week['Neutor FR stadteinwärts']
            )
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [18/May/2023 15:50:22] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2023 15:50:22] "GET /assets/style.css?m=1684158671.2483714 HTTP/1.1" 304 -
127.0.0.1 - - [18/May/2023 15:50:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2023 15:50:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/May/2023 15:50:22] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [18/May/2023 15:50:22] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


In [44]:
print(df_neutor_last_week)

         Datum  Neutor  Neutor FR stadteinwärts  Neutor FR stadtauswärts   
128 2020-05-08  8819.0                   4641.0                   4178.0  \
129 2020-05-09  7734.0                   3976.0                   3758.0   
130 2020-05-10  3792.0                   1909.0                   1883.0   
131 2020-05-11  6827.0                   3374.0                   3453.0   
132 2020-05-12  7782.0                   3850.0                   3932.0   
133 2020-05-13  8366.0                   4186.0                   4180.0   
134 2020-05-14  8673.0                   4302.0                   4371.0   

     Wochentag  
128     Friday  
129   Saturday  
130     Sunday  
131     Monday  
132    Tuesday  
133  Wednesday  
134   Thursday  


In [4]:
fig = go.Figure()

fig = fig.add_trace(
            go.Bar(
                name='SE',
                x=df_neutor_last_week['Datum'],
                y=df_neutor_last_week['Neutor FR stadteinwärts'],
                offsetgroup=0
            )
    )
fig.add_trace(
        go.Bar(
                name='SA',
                x=df_neutor_last_week['Datum'],
                y=df_neutor_last_week['Neutor FR stadtauswärts'],
                offsetgroup=0,
                base=df_neutor_last_week['Neutor FR stadteinwärts']
        )
)

In [56]:
import plotly.graph_objects as go
import plotly.express as px
import dash
from dash import Dash, dcc, html, Input, Output

df_neutor_weekday = neutor_weekday()
df_neutor_last_week = neutor_last_week()

app = Dash(__name__)

app.layout = html.Div(children=[
    html.Div([
        dcc.Graph(id='graph-with-slider'),
        html.Div([
            dcc.Dropdown(
                options=[
                    {'label': 'Neutor', 'value': 'Neutor'},
                    {'label': 'Neutor EW', 'value': 'Neutor FR stadteinwärts'},
                    {'label': 'Neutor AW', 'value': 'Neutor FR stadtauswärts'}
                ],
                value='Neutor',
                id='yaxis-column'
            )
        ])
    ]),
    html.Div([
        dcc.Graph(id='second-graph')  # Neuer Graph
    ]),
])

@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('yaxis-column', 'value'))
def update_figure(selected_column):
    fig = px.box(df_neutor_weekday, x='Wochentag', y=selected_column, points='all')
    return fig

@app.callback(
    Output('second-graph', 'figure'))  # Callback-Methode für den zweiten Plot
def update_second_figure():
    fig = go.Figure()

    fig.add_trace(
        go.Bar(
            name='SE',
            x=df_neutor_last_week['Datum'],
            y=df_neutor_last_week['Neutor FR stadteinwärts'],
            offsetgroup=0
        )
    )
    fig.add_trace(
        go.Bar(
            name='SA',
            x=df_neutor_last_week['Datum'],
            y=df_neutor_last_week['Neutor FR stadtauswärts'],
            offsetgroup=0,
            base=df_neutor_last_week['Neutor FR stadteinwärts']
        )
    )
    return fig

if __name__ == '__main__':
    app.run_server(debug=False)


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050
Press CTRL+C to quit
127.0.0.1 - - [15/May/2023 14:31:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2023 14:31:14] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2023 14:31:14] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2023 14:31:14] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2023 14:31:14] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2023 14:31:14] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -


In [ ]:
# Import packages
from dash import Dash, html, dash_table, dcc, callback, Output, Input
import pandas as pd
import plotly.express as px

# Incorporate data
df_neutor_weekday = neutor_weekday()
df_neutor_last_week = neutor_last_week()

# Initialize the app
app = Dash(__name__)

app.layout = html.Div(children=[
    html.Div([
        dcc.Graph(id='graph-with-slider'),
        html.Div([
            dcc.Dropdown(
                options=[
                    {'label': 'Neutor', 'value': 'Neutor'},
                    {'label': 'Neutor EW', 'value': 'Neutor FR stadteinwärts'},
                    {'label': 'Neutor AW', 'value': 'Neutor FR stadtauswärts'}
                ],
                value='Neutor',
                id='yaxis-column'
            )
        ])
    ]),
    html.Div([ 
        dcc.Graph(id='tip-graph'),
        dcc.Graph(id='second-graph'),  # Neuer Graph
        ]),
    ])

# App layout
app.layout = html.Div([
    html.Div(children='My First App with Data, Graph, and Controls'),
    html.Hr(),
    dcc.RadioItems(options=['pop', 'lifeExp', 'gdpPercap'], value='lifeExp', id='controls-and-radio-item'),
    dash_table.DataTable(data=df.to_dict('records'), page_size=6),
    dcc.Graph(figure={}, id='controls-and-graph')
])

# Add controls to build the interaction
@app.callback(
    Output('graph-with-slider', 'figure'),
    Input('yaxis-column', 'value'))
def update_figure(selected_column):
    fig = px.box(df_neutor_weekday, x='Wochentag', y=selected_column, points='all')

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)